# 4 - Amazon Bedrock Agents

This notebook provides sample code for building an Agent with Amazon Bedrock Agents.

## Use Case
Now that we have a data source and RAG setup, we also need some useful actions or tools that can perform basic booking in this app. For this we will create a Hotel Booking assistant that allows customers to create, delete or get reservation information. The architecture looks as following:

![Agent Architecture](images/MM-Agents.gif)

## Notebook walk-through

In this notebook we will:
- Choose our Agent's underlying foundation model
- Create a dynamoDB table to store the reservation details
- Create a lambda function that handles the restaurant bookings
- Create an agent
- Create an action group and associate it with the agent
- Associate the knowledge base we have created to the agent
- Test the agent invocation

In the next lab, we will add a Knowledge Base to our agent and work with Prompt Attributes to provide extra information to an agent invocation call


## Prerequisite

This notebook requires permissions to:
- create and delete Amazon IAM roles
- create lambda functions
- create dynamoDB tables
- access Amazon Bedrock

If running on SageMaker Studio, you should add the following managed policies to your role:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess
- AmazonDynamoDBFullAccess

<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure to enable <b>Anthropic Haiku 3</b>, <b>Amazon Nova Pro</b>, and <b>Amazon Titan Embedding v2</b> model access in Amazon Bedrock Console, as this notebook will use these models for testing.
</div>


## Setup
Before running the rest of this notebook, you'll need to run the cells below to ensure necessary libraries are installed and set up correctly:

In [ ]:
import logging
import time
import uuid
import boto3
import json
import pandas as pd
import ipywidgets as widgets

from agent import create_agent_role, create_lambda_role
from agent import create_dynamodb, create_lambda, invoke_agent_helper

In [ ]:
%store -r

In [ ]:
#Clients
boto3.setup_default_session(region_name=region_name)

s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
bedrock_agent_client = session.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

account_id = sts_client.get_caller_identity().get('Account')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

region_name, account_id

### Setting up Agent's information

We will now set the variables that define our agent:

- **agent_name**: provides the name of the agent to be created, in this case `booking-agent`
- **agent_description**: the description of the agent used to display the agents list on the console. This description is **not** part of the agent's prompts
- **agent_instruction**: the instructions of what the agent should and should not do. This description is part of the agent's prompt and is used during the agent's invocation
- **agent_action_group_name**: the action group name used on the definition of the agent's action, in this case `HotelBookingsActionGroup`.
- **agent_action_group_description:**: the description of the action group name used on the UI to list the action groups. This description is **not** used by the agent's prompts

In [ ]:
suffix = f"{region_name}-{account_id}"
agent_name = "hotel-booking-agent"
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f"AmazonBedrockExecutionRoleForAgents_{agent_name}"

agent_description = "Agent is in charge of hotel bookings"
agent_instruction = """
You are a ABC Grand Hotel Booking agent, one of the key chain of hotels Hosting Amazon Reinvent 2024 helping clients retrieve information about the hotel,
create a new booking or cancel an existing booking
"""

agent_action_group_description = """
Actions for getting Hotel booking information, create a new hotel booking or delete an existing booking"""

agent_action_group_name = "HotelBookingsActionGroup"

### Select Foundation Model
Select the underline model of your agent. You can find more information about the supported foundation models [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-supported.html).

In [ ]:
agent_foundation_model = "us.amazon.nova-pro-v1:0"

### Creating DynamoDB table

Let's now create an [Amazon DynamoDB](https://aws.amazon.com/dynamodb/) table called `hotel_bookings`. This table will store information about the reservations, including a `booking_id`, reservation `date`, the `name` of the person doing the reservation, the `hour` of the reservation and the number of guests as `num_guests`. To do so, we use the `create_dynamodb` function from the `agent.py` file. This function will support the creation of the table and its requirements (IAM roles and permissions).

In [ ]:
table_name = 'hotel_bookings'
create_dynamodb(table_name)

### Creating Lambda Function

Next we will create the [AWS Lambda](https://aws.amazon.com/lambda/) function that executes the actions for our agent. This lambda function will have 3 actions:
* ```get_booking_details(booking_id)```: returns the details of a booking based on the booking id
* ```create_booking(date, name, hour, num_guests)```: creates a new booking for the restaurant
* ```delete_booking(booking_id)```: deletes an existent booking based on the booking id


The `lambda_handler` receives the `event` from the agent and the `event` contains information about the `function` to be executed and its `parameters`. 

A `functionResponse` is returned by the lambda function with the response body having a `TEXT` field.

You can find more information on how to set your agent lambda function [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html).

Let's first write the code of the lambda function to the `lambda_function.py` file

In [ ]:
code = f"""\
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('{table_name}')  # Dynamically inject the value of tab


def get_named_parameter(event, name):
    \"\"\"
    Get a parameter from the lambda event
    \"\"\"
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_booking_details(booking_id):
    \"\"\"
    Retrieve details of a hotel booking

    Args:
        booking_id (string): The ID of the booking to retrieve
    \"\"\"
    try:
        response = table.get_item(Key={{'booking_id': booking_id}})
        if 'Item' in response:
            return response['Item']
        else:
            return {{'message': f'No booking found with ID {{booking_id}}'}}
    except Exception as e:
        return {{'error': str(e)}}




def create_booking(date, name, hour, num_guests):
    \"\"\"
    Create a new hotel booking

    Args:
        date (string): The date of the booking
        name (string): Name to identify your reservation
        hour (string): The hour of the booking
        num_guests (integer): The number of guests for the booking
    \"\"\"
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={{
                'booking_id': booking_id,
                'date': date,
                'name': name,
                'hour': hour,
                'num_guests': num_guests
            }}
        )
        return {{'booking_id': booking_id}}
    except Exception as e:
        return {{'error': str(e)}}




def delete_booking(booking_id):
    \"\"\"
    Delete an existing hotel booking

    Args:
        booking_id (str): The ID of the booking to delete
    \"\"\"
    try:
        response = table.delete_item(Key={{'booking_id': booking_id}})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {{'message': f'Booking with ID {{booking_id}} deleted successfully'}}
        else:
            return {{'message': f'Failed to delete booking with ID {{booking_id}}'}}
    except Exception as e:
        return {{'error': str(e)}}




def lambda_handler(event, context):
    # Get the action group used during the invocation of the lambda function
    actionGroup = event.get('actionGroup', '')

    # Name of the function that should be invoked
    function = event.get('function', '')

    # Parameters to invoke function with
    parameters = event.get('parameters', [])




    if function == 'get_booking_details':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(get_booking_details(booking_id))
            responseBody = {{'TEXT': {{'body': json.dumps(response)}}}}
        else:
            responseBody = {{'TEXT': {{'body': 'Missing booking_id parameter'}}}}




    elif function == 'create_booking':
        date = get_named_parameter(event, "date")
        name = get_named_parameter(event, "name")
        hour = get_named_parameter(event, "hour")
        num_guests = get_named_parameter(event, "num_guests")




        if date and hour and num_guests:
            response = str(create_booking(date, name, hour, num_guests))
            responseBody = {{'TEXT': {{'body': json.dumps(response)}}}}
        else:
            responseBody = {{'TEXT': {{'body': 'Missing required parameters'}}}}




    elif function == 'delete_booking':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(delete_booking(booking_id))
            responseBody = {{'TEXT': {{'body': json.dumps(response)}}}}
        else:
            responseBody = {{'TEXT': {{'body': 'Missing booking_id parameter'}}}}




    else:
        responseBody = {{'TEXT': {{'body': 'Invalid function'}}}}




    action_response = {{
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {{
            'responseBody': responseBody
        }}
    }}




    function_response = {{'response': action_response, 'messageVersion': event['messageVersion']}}
    print("Response: {{}}".format(function_response))




    return function_response
"""


# Write the dynamically generated content to a file
with open("lambda_function.py", "w") as file:
    file.write(code)


print("lambda_function.py has been successfully created ")


Next we create the function requirements for IAM role and policies using the support function `create_lambda_role` and create the lambda using the support function `create_lambda` both from the `agent.py` file

In [ ]:
lambda_iam_role = create_lambda_role(agent_name, table_name)
lambda_function_name = f"{agent_name}-lambda"
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

### Creating Agent

Now that we have created the dynamoDB table and lambda function, let's create our Agent. 

To do so, we first need to create an agent role and its required policies. Let's do so using the `create_agent_role` function from the `agent.py` file.

In [ ]:
agent_role = create_agent_role(agent_name, agent_foundation_model)
agent_role

With the Agent IAM role created, we can now use the boto3 function [`create_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent.html) to create our agent. 

On the agent creation, all you need to provide is the agent name, foundation model and instruction. We will associate an action group to the agent once it has been created.

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

Now that our agent has been created, we will retrieve the `agentId`. It will be used to associate the action group to the agent in our next step.

In [ ]:
agent_id = response['agent']['agentId']
print(f"The agent ID is: {agent_id}")

We can also go to the Console at Bedrock > Agents and see our agent should be coming up
![images/agent/agent_start.png](images/agent/agent_start.png)

#### Create Agent Action Group

now that we have created the agent, let's create an [Action Group](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html) and associate with the agent. The action group will allow our agent to execute the booking tasks. To do so, we will "inform" our agent about the existent functionalities using a [function schema](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html) defined in `JSON` format.

The function schema requires the function `name`, `description` and `parameters` to be provided. Each parameter has a parameter name, description, type and a boolean flag indicating if the parameter is required.

Let's define the functions `JSON` as `agent_functions`

In [ ]:
agent_functions = [
    {
        "name": "get_booking_details",
        "description": "Retrieve details of a the hotel booking",
        "parameters": {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string",
            }
        },
    },
    {
        "name": "create_booking",
        "description": "Create a new hotel booking",
        "parameters": {
            "date": {
                "description": "The date of the booking in the format YYYY-MM-DD",
                "required": True,
                "type": "string",
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string",
            },
            "hour": {
                "description": "The hour of the booking in the format HH:MM",
                "required": True,
                "type": "string",
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer",
            },
        },
    },
    {
        "name": "delete_booking",
        "description": "Delete an existing hotel booking",
        "parameters": {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string",
            }
        },
    },
]

Now we can use the [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function from the boto3 SDK to create the action group.

In [ ]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion="DRAFT",
    actionGroupExecutor={"lambda": lambda_function["FunctionArn"]},
    actionGroupName=agent_action_group_name,
    functionSchema={"functions": agent_functions},
    description=agent_action_group_description,
)
agent_action_group_response

#### Allowing bedrock to invoke lambda function

The last requirement is to add the [resource-based policy](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-permissions.html#agents-permissions-lambda) to allow bedrock to invoke the action group lambda function.

In [ ]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client("lambda")
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f"allow_bedrock_{agent_id}_2",
        Action="lambda:InvokeFunction",
        Principal="bedrock.amazonaws.com",
        SourceArn=f"arn:aws:bedrock:{region_name}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

#### Preparing agent

Before invoking the agent we need to prepare it. Preparing your agent will package all its components, including the security configurations. It will bring the agent into a state where it can be tested in runtime. We will use the [`prepare_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/prepare_agent.html) function from the boto3 sdk to prepare our agent.

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

# Pause to make sure agent is prepared
time.sleep(30)

### Invoking Agent

Now that our Agent is ready to be used, let's test it. To do so we will use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

To invoke an agent, you have to refer to its alias. You can create a new alias, or you can use the test alias to invoke your `DRAFT` agent. The test alias used to invoke the draft agent is `TSTALIASID` and it will work with any agent. 


We will use the support function called `invoke_agent_helper` from the `agents.py` support file to allow us to invoke the agent with or without trace enabled and with or without session state.

In [ ]:
alias_id = 'TSTALIASID'

In [ ]:
%%time

session_id = str(uuid.uuid1())
query = "Hi, my name is John Doe. I want to create a hotel booking for 2 people, with check-in time of 8pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)


## Associate Knowledge Base to Agent

- Use the already created Knowledge Base and its pre-requirements (including OpenSearch Servelless Collection and Indexes)
- Update Agent IAM role to allow for Knowledge Base access
- Associate Knowledge Base with Hotel Booking Agent
- Test Agent invocation with Knowledge Base access

### Updating Agent role to allow Knowledge Base Retrieve and Retrieve and Generate queries

Now that our Knowledge Base is working, we will associate it with the agent. To do so, we first need to update the agent role to allow for retrieval from context from our knowledge base

In [ ]:
iam_client = boto3.client('iam')

In [ ]:
kb_policies_statements = [
    {
        "Sid": "QueryKB",
        "Effect": "Allow",
        "Action": ["bedrock:Retrieve", "bedrock:RetrieveAndGenerate"],
        "Resource": [
            f"arn:aws:bedrock:{region_name}:{account_id}:knowledge-base/{kb_id}"
        ],
    }
]
bedrock_agent_kb_policy_statement = {
    "Version": "2012-10-17",
    "Statement": kb_policies_statements,
}
bedrock_agent_kb_policy_json = json.dumps(bedrock_agent_kb_policy_statement)
kb_policy_name = f"{agent_name}-kb-{kb_id}"
agent_kb_policy = iam_client.create_policy(
    PolicyName=kb_policy_name,
    PolicyDocument=bedrock_agent_kb_policy_json,
)

In [ ]:
iam_client.attach_role_policy(
    RoleName=agent_role["Role"]["RoleName"],
    PolicyArn=agent_kb_policy["Policy"]["Arn"]
)

#### Associate Knowledge Base with Agent

Finally, we can associate the new knowledge base with the agent using the [`AssociateAgentKnowledgeBase`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/associate_agent_knowledge_base.html) API from boto3

In [ ]:
time.sleep(10)
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion="DRAFT",
    description="Access the knowledge base when customers ask about the information about the hotel",
    knowledgeBaseId=kb_id,
    knowledgeBaseState="ENABLED",
)

#### Preparing Agent

after updating our agent, we need to prepare it again to package all its new components

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

# Pause to make sure agent is prepared
intermediate_agent_status = ['CREATING', 'PREPARING', 'UPDATING', 'VERSIONING']
while bedrock_agent_client.get_agent(agentId=agent_id)['agent']['agentStatus'] in intermediate_agent_status:
    time.sleep(10)


#### Invoking Agent

Now that our Agent has been updated, let's test it again. To do so we will again use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

We will use the same support function called `invoke_agent_helper` to allow us to invoke the agent with or without trace enabled and with or without session state.

Now we can test it by asking a question where the answer is available in the knowledge base documents.

In [ ]:
%%time

session_id = str(uuid.uuid1())
query = """I want to book a room in ABC grand hotel, but I know my booking means i am investing in it.
So lets first find out ABC Grand'financial performance in 2023."""
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

In [ ]:
%%time

session_id = str(uuid.uuid1())
query = """What was the EBITDA in 2023 and 2022 for ABC Grand in millions?"""
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

## Testing the agent

Now that we've created the agent and attached the knowledge base to it, we will use the runtime client to invoke the agent with different queries. The boto3 sdk for agent is divided into two clients: `bedrock-agent` and `bedrock-agent-runtime`. The `bedrock-agent` client is responsible for the functionalities to create, update, delete and/or prepare an agent or a knowledge base. While the `bedrock-agent-runtime` is responsible for invoke an agent (with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API) and retrieve documents from a knowledge base (with the [`retrieve`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [`retrieve_and_generate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) APIs). This notebook will focus on the runtime invocation of the agent created in the previous notebooks. We will use the `invoke_agent` API.


In [ ]:
dynamodb = boto3.resource('dynamodb')

#### Create support selectAllFromDynamoDB function

We will also create the support function called `selectAllFromDynamoDB` to select all data in the dynamoDB table `restaurant_bookings`. This function will be used to validate our agent's behaviour.

In [ ]:
def selectAllFromDynamodb():
    # Get the table object
    table = dynamodb.Table(table_name)

    # Scan the table and get all items
    response = table.scan()
    items = response['Items']

    # Handle pagination if necessary
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    items = pd.DataFrame(items)
    return items

In [ ]:
# test function invocation
items = selectAllFromDynamodb()
items

### Invoking agent with a new session id
Let's first use the `InvokeAgent` function to query the Knowledge Base with the agent without previous context

In [ ]:
%%time

session_id = str(uuid.uuid1())
query = "Are dogs allowed to the ABC Grand hotel?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

### Invoking agent with existent session id

Next we can use the context to ask a follow up question. To do so, we use the same `session_id` with the `invokeAgent` function

In [ ]:
%%time

query = "what about cats?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

As you can see, the agent knows that we are talking about the pet policy.

### Invoking agent using action group

Now let's use our agent to make a reservation. By doing so, we will require the agent to execute an action from our action group to create a new reservation.

In [ ]:
%%time

session_id = str(uuid.uuid1())
query = """I want to book a room in ABC grand hotel, but
I know my booking means I am investing in it. So lets first find out
how has revenue grown from 2022 to 2023 for ABC Grand Hotel?"""

response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

In [ ]:
%%time

query = "Awesome! I think thats postive. Can you create a booking for 4 people under name 'Reinvent Booking' with a checkin at 9pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Let's double check that the data was properly added to the dynamoDB table

In [ ]:
selectAllFromDynamodb()

Great! We've used our agent to do create a reservation. However, often when booking rooms in hotel we are already logged in to a system that know our names. How great would it be if our agent would know it as well?

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [promptSessionAttributes](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [ ]:
%%time

session_id = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm on the 6th of May 2024"
session_state = {
    "promptSessionAttributes": {
        "name": "Reinvent Booking Login"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Again let's validate the the correct data was added to dynamoDB

In [ ]:
selectAllFromDynamodb()

We can also validate the data using our agent's session information since the agent knows the booking id to invoke the get booking details function

In [ ]:
%%time

query = "Get the details for the last booking created"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

#### Deleting booking created

Let's also test the delete booking functionality by deleting the last created booking id

In [ ]:
%%time

query = "I want to delete the booking."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

And we confirm that the booking has also been deleted from the DynamoDB table:

In [ ]:
selectAllFromDynamodb()

## Test the agent with Bedrock console (optional)

Navigate to the [Amazon Bedrock > Agents > hotel-booking-agent](https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/agents) console and test the agent chat experience.